<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/Stcok_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q bta-lib
!pip install -q ta
!pip install -q yfinance

In [23]:
import yfinance as yf
import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.trend import ADXIndicator

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 500)

In [24]:
xu030 = [
    "AKBNK", "AKSEN", "ALARK", "ARCLK", "ASELS", "BIMAS", "EKGYO", "ENKAI", "EREGL", "FROTO",
    "GARAN", "GUBRF", "HEKTS","KRDMD","KCHOL", "KOZAL", "KOZAA", "ODAS", "PGSUS", "SAHOL",
    "SASA", "PETKM", "SISE", "TAVHL", "THYAO", "TUPRS", "TOASO", "TCELL", "ISCTR", "YKBNK"]

xu100 = [
    "ADESE", "AGHOL", "AKBNK", "AKSA", "AKSEN", "ALGYO", "ALARK", "ALKIM", "AEFES", "ARCLK",
    "ARDYZ", "ASELS", "ASTOR", "AYDEM", "BASGZ", "BERA", "BIMAS", "BIOEN", "BRYAT", "BRISA",
    "CCOLA", "CANTE", "CEMAS", "CIMSA", "DOHOL", "DOAS", "EGEEN", "ECILC", "EKGYO", "ENJSA",
    "ENKAI", "ERBOS", "EREGL", "ESEN", "FROTO", "GENIL", "GESAN", "GLYHO", "GOZDE", "GUBRF",
    "SAHOL", "HEKTS", "INDES", "IPEKE", "ISDMR", "ISFIN", "ISGYO", "ISMEN", "IZMDC", "KRDMD",
    "KARSN", "KARTN", "KERVT", "KRVGD", "KCHOL", "KORDS", "KOZAL", "KOZAA", "LOGO", "MAVI",
    "MGROS", "NTHOL", "ODAS", "OTKAR", "OYAKC", "PARSN", "PGSUS", "PETKM", "QUAGR", "RTALB",
    "SARKY", "SASA", "SELEC", "SKBNK", "SOKE", "SOKM", "TAVHL", "TKFEN", "TKNSA", "TOASO",
    "TRGYO", "TCELL", "TUPRS", "THYAO", "TTKOM", "TTRAK", "GARAN", "HALKB", "ISCTR", "TSKB",
    "TURSG", "SISE", "VAKBN", "ULKER", "VERUS", "VESBE", "VESTL", "YKBNK", "YATAS", "ZRGYO",
    "ZOREN"
]

# url = 'https://en.m.wikipedia.org/wiki/Nasdaq-100'
# nasdaq100 = pd.read_html(url, attrs={'id': "constituents"})[0]
# nasdaq100 = nasdaq100["Ticker"].to_list()

In [25]:
def create_indicators(data):
  # Below 30 is oversold, upper 70 is overbought
  data["RSI"] = btalib.rsi(data["Close"], period=14).df

  #If little period conflicts with the bigger one it means buy else sell.
  data["SMA9"] = btalib.sma(data['Close'], period=9).df
  data["SMA20"] = btalib.sma(data['Close'], period=20).df

  #A reading below 20 generally represents an oversold market and a reading above 80 an overbought market. Look only STOCH-D.
  data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
  data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

  #When the MACD line crosses above the signal line, it generates a buy signal. This indicates a potential bullish trend reversal.
  #Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
  data["MACD"] = btalib.macd(data['Close']).df["macd"]
  data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
  data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]

  #Buy Signal: When the price of the stock touches or crosses below the lower Bollinger Band, it can be considered a buy signal.
  #This suggests that the stock may be oversold, and a potential price rebound is expected.
  #Sell Signal: When the price of the stock touches or crosses above the upper Bollinger Band, it can be considered a sell signal.
  #This indicates that the stock may be overbought, and a potential price correction is expected.
  data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
  data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
  data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']

  #When the current price of the stock falls below a certain number of standard deviations (GET THE MIN AND MAX VALUE OF THE COLUMN AND MAKE IT THE THRESHOLD)
  #from the mean, it generates a buy signal. This suggests that the price has deviated significantly from the average and may present a buying opportunity.
  data["STDEV"] = data["Close"].rolling(window=10).std()

  #Generate a buy signal when the price crosses above the PSAR.
  #This indicates a potential reversal in the downward trend and suggests a buying opportunity.
  data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()

  #Williams
  #Use the Williams Percent Range (%R) formula to calculate the %R values based on historical price data.
  #The %R values range from -100 to 0, where values close to -100 indicate oversold conditions and values close to 0 indicate overbought conditions.
  #Generate a buy signal when the %R value crosses above a certain threshold (e.g., -80) from below.
  #This indicates a potential reversal from oversold conditions and suggests a buying opportunity.
  data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()

  #The difference between these two lines indicates whether there is overbought (a positive number) or oversold (a negative number).
  data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()

  #obv
  data["OBV"] = VolumePriceTrendIndicator(close=data['Close'], volume=data['Volume']).volume_price_trend()

  #CCI
  data['CCI'] = CCIIndicator(close=data['Close'], low=data["Low"], high=data["High"], window=14).cci()

  #ROC
  data['ROC'] = ROCIndicator(close=data['Close'], window=5).roc()

  #BULLBEARPOWER
  data['BULL'] = data['High'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2
  data['BEAR'] = data['Low'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2

  #ADX
  adx_indicator = ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)
  data['ADX'] = adx_indicator.adx()
  data['+DI'] = adx_indicator.adx_pos()
  data['-DI'] = adx_indicator.adx_neg()

  data = data.dropna()
  data = data.reset_index()
  return data

def score(data):
  scores = []

  for row in range(data.shape[0]):
    #MAX 15.25
    #MIN -15.25

    min_score = -15.25
    max_score = 15.25
    score = 0

    if data["RSI"][row] < 30:
      score += 1
    elif data["RSI"][row] > 70:
      score+=-1
    # else:
    #   score+=(100-int(data["RSI"][row]))/70

    if data["SMA9"][row] > data["SMA20"][row] and data["Close"][row] > data["SMA9"][row]:
      score += 1
    else:
      score += -1

    if data["STOCH-D"][row] <= 20:
      score += 1
    elif data["STOCH-D"][row] >= 80:
      score += -1

    if data["MACD"][row] > data["SIGNAL"][row]:
      score+=1
    if row > 0 and data["MACD"][row] > data["SIGNAL"][row] and data["HISTOGRAM"][row-1] < 0 and data["HISTOGRAM"][row] > 0:
      score+=1
    if row > 0 and data["MACD"][row] < data["SIGNAL"][row] and data["HISTOGRAM"][row-1] > 0 and data["HISTOGRAM"][row] > 0:
      score+=-1
    elif data["MACD"][row] < data["SIGNAL"][row]:
      score+=-1

    if data["BB-LOWER"].iloc[row] > data["Close"].iloc[row]:
        score += 1
    elif data["BB-MID"].iloc[row] < data["Close"].iloc[row] and data["BB-LOWER"].iloc[row] - data["Close"].iloc[row] < data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score += 0.25
    elif data["BB-MID"].iloc[row] > data["Close"].iloc[row] and data["BB-LOWER"].iloc[row] - data["Close"].iloc[row] > data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score -= 0.25
    elif data["BB-UPPER"].iloc[row] < data["Close"].iloc[row]:
        score -= 1

    if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
      score+=1
    else:
      score+=-1

    if data["PSAR"][row] < data["Close"][row]:
      score+=1
    else:
      score+=-1

    if data["WILLIAMS"][row] < -80:
      score+=1
    elif data["WILLIAMS"][row] > -20:
      score+=-1

    if data["AROON"][row] > 0:
      score+=-1
    else:
      score+=1

    if row > 0 and data["OBV"][row-1] < data["OBV"][row] and data["Close"][row] > data["Close"][row-1]:
      score+=1
    elif row > 0 and data["OBV"][row-1] > data["OBV"][row] and data["Close"][row] < data["Close"][row-1]:
      score+=-1

    if row > 0 and (data['CCI'][row] > -100) & (data['CCI'][row-1] <= -100):
      score+=1
    elif row > 0 and (data['CCI'][row] < 100) & (data['CCI'][row-1] >= 100):
      score-=1

    if data["ROC"][row] > 0:
      score+=1
    elif data["ROC"][row] < 0:
      score-=1

    if data["BULL"][row] > 0:
      score+=1
    elif data["BEAR"][row] < 0:
      score-=1

    if (data['ADX'][row] > 25) and (data['+DI'][row] > data['-DI'][row]):
      score+=1
    elif (data['ADX'][row] > 50) and (data['+DI'][row] < data['-DI'][row]):
      score-=1

    #normalized score between 0-100
    normalized_score = ((score - min_score) / (max_score - min_score)) * (100 - 0) + 0
    scores.append(normalized_score)

  data["SCORE"] = scores
  return data

def save_to_excel(stock, data):
  output_file = stock + ":" + start + ":" + end + '.xlsx'
  writer = pd.ExcelWriter(output_file)
  data.to_excel(writer, index=True, sheet_name=stock)
  writer.save()

def simulation(stock, data, balance, upper_score, lower_score):
  stock_balance = 0
  transactions = []

  for row in range(data.shape[0]):
    if data["SCORE"][row] >= upper_score and balance > data["Close"][row]:
      stock_balance = balance / data["Close"][row]
      balance-= stock_balance*data["Close"][row]
      transactions.append([stock, "BUY", data["Date"][row], data["Close"][row], stock_balance, balance])

    elif data["SCORE"][row] <= lower_score and stock_balance > 1:
      balance+=stock_balance*data["Close"][row]
      stock_balance-=stock_balance
      transactions.append([stock,"SELL", data["Date"][row], data["Close"][row], stock_balance, balance])

  transactions_df = pd.DataFrame(transactions, columns=["STOCK","B/S", "DATE", "PRICE", "STOCK BALANCE", "TRY BALANCE"])
  return transactions_df


In [ ]:
start="2023-01-01"
end="2023-07-01"

buy_stocks = []
sell_stocks = []
simulation_stocks = pd.DataFrame()
lookup_table = ["AKBNK", "BIMAS", "ISCTR", "PGSUS", "GARAN", "ARCLK"]
portfolio = []

buy_threshold = 70
sell_threshold = 30

for stock in xu100:
  try:
    data = yf.download(stock+".IS", start=start, end=end)
    data = create_indicators(data)
    data = score(data)

    if stock in lookup_table:
      # save_to_excel(stock, data)
      portfolio.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item(),
                        "BUY" if data.tail(1)["SCORE"].item() > buy_threshold else "SELL" if data.tail(1)["SCORE"].item() < sell_threshold else "HOLD"])

    if data.tail(1)["SCORE"].item() > buy_threshold:
      buy_stocks.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item(),
                         "BUY" if data.tail(1)["SCORE"].item() > buy_threshold else "SELL" if data.tail(1)["SCORE"].item() < sell_threshold else "HOLD"])
      simulation_df = simulation(stock, data, balance=2000, upper_score=buy_threshold, lower_score=sell_threshold)
      simulation_stocks = pd.concat([simulation_stocks, simulation_df])

    elif data.tail(1)["SCORE"].item() < sell_threshold:
      sell_stocks.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item(),
                          "BUY" if data.tail(1)["SCORE"].item() > buy_threshold else "SELL" if data.tail(1)["SCORE"].item() < sell_threshold else "HOLD"])

  except:
    print("Data error at the stock named: " + stock)

In [89]:
portfolio = pd.DataFrame(portfolio, columns=["Date","Stock", "Price","Score", "B/S/H"])
portfolio = portfolio.sort_values(by='Score', ascending=False)
portfolio

,Date,Stock,Price,Score,B/S/H
0,2023-06-16,AKBNK,18.89,67.21,HOLD
2,2023-06-16,BIMAS,176.90,67.21,HOLD
5,2023-06-16,ISCTR,13.42,60.66,HOLD
3,2023-06-16,PGSUS,589.30,57.38,HOLD
4,2023-06-16,GARAN,30.08,57.38,HOLD
1,2023-06-16,ARCLK,111.50,54.10,HOLD


In [91]:
buy_stocks = pd.DataFrame(buy_stocks, columns=["Date","Stock", "Price","Score", "B/S/H"])
buy_stocks = buy_stocks.sort_values(by='Score', ascending=False)
buy_stocks

,Date,Stock,Price,Score,B/S/H
0,2023-06-16,CCOLA,247.10,73.77,BUY
1,2023-06-16,SAHOL,43.98,70.49,BUY
2,2023-06-16,IZMDC,5.72,70.49,BUY
3,2023-06-16,MAVI,63.95,70.49,BUY


In [92]:
sell_stocks = pd.DataFrame(sell_stocks, columns=["Date","Stock", "Price","Score", "B/S/H"])
sell_stocks

,Date,Stock,Price,Score,B/S/H


In [93]:
simulation_stocks.where(simulation_stocks["STOCK"] == "CCOLA").dropna()

,STOCK,B/S,DATE,PRICE,STOCK BALANCE,TRY BALANCE
0,CCOLA,BUY,2023-02-24,184.40,10.85,0.00
1,CCOLA,SELL,2023-05-22,250.70,0.00,2719.09
2,CCOLA,BUY,2023-06-12,255.10,10.66,0.00


In [94]:
test_dfs = pd.DataFrame()

for stock in xu030:
  data = yf.download(stock+".IS", start, end)
  data = create_indicators(data)
  data = score(data)
  simulation_df = simulation(stock, data, balance=2000, upper_score=buy_threshold, lower_score=sell_threshold)
  test_dfs = pd.concat([test_dfs, simulation_df])

test_dfs

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,STOCK,B/S,DATE,PRICE,STOCK BALANCE,TRY BALANCE
0,AKBNK,BUY,2023-02-27,17.32,115.47,0.00
1,AKBNK,SELL,2023-05-17,15.58,0.00,1799.08
2,AKBNK,BUY,2023-06-14,18.37,97.94,0.00
0,AKSEN,BUY,2023-02-28,36.12,55.37,0.00
0,ALARK,BUY,2023-04-07,70.00,28.57,0.00
0,ARCLK,BUY,2023-02-28,119.40,16.75,0.00
0,ASELS,BUY,2023-03-06,55.70,35.91,0.00
0,BIMAS,BUY,2023-02-23,133.60,14.97,0.00
0,EKGYO,BUY,2023-03-01,8.43,237.25,0.00
0,ENKAI,BUY,2023-04-04,31.52,63.45,0.00
